In [1]:
# import sys/os
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-3])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions/utils")

# add integration folder to current working path
sys.path.append(parent_directory+"/subfunctions/integration")

# Overview

In the following demos we illustrate how to compute the Polar Decomposition (PD) of the gradient of the flow map $ \nabla F_{t_0}^t(\mathbf{x}_0) $ on a general two dimensional unsteady velocity dataset. In our case we chose the AVISO dataset which is used throughout the demos on advective barriers in 2D. The notebook is structured as follows:

1. Import data from the file 'AVISO.mat' stored in the folder 'data/AVISO'.
2. Define computational parameters (such as the number of cores) and variables.
3. Interpolate velocity from the (discrete) gridded data.
4. Compute gradient of flow map $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $.
5. PD on $ \mathbf{\nabla F}_{t_0}^{t_N}(\mathbf{x}_0) $

# Import data

In [2]:
# import scipy
import scipy.io as sio

#Import velocity data from file in data-folder
mat_file = sio.loadmat('../../../data/Aviso/AVISO.mat')

U = mat_file['u']
V = mat_file['v']
x = mat_file['x']
y = mat_file['y']
time_data = mat_file['t']

# Computational parameters and data

Here we define the computational parameters and the data.

In [3]:
# import numpy
import numpy as np

# Number of cores for parallel computing
Ncores = 6

# Time resolution of data
dt_data = time_data[0, 1]-time_data[0,0]

# Periodic boundary conditions
periodic_x = False
periodic_y = False
periodic_t = False
periodic = [periodic_x, periodic_y, periodic_t]

# Unsteady velocity field
bool_unsteady = True

# Defined domain
defined_domain = np.isfinite(U[:,:,0]).astype(int)

## Compute meshgrid of dataset
X, Y = np.meshgrid(x, y)

## Resolution of meshgrid
dx_data = X[0,1]-X[0,0]
dy_data = Y[1,0]-Y[0,0]

delta = [dx_data, dy_data]

# Velocity Interpolation

In order to evaluate the velocity field at arbitrary locations and times, we interpolate the discrete velocity data. The interpolation with respect to time is always linear. The interpolation with respect to space can be chosen to be "cubic" or "linear". Default value is "cubic".

In [4]:
# Import interpolation function for unsteady flow field
from ipynb.fs.defs.Interpolant import interpolant_unsteady

# Set nan values to zero (in case there are any) so that we can apply interpolant. 
# Interpolant does not work if the array contains nan values. 
U[np.isnan(U)] = 0
V[np.isnan(V)] = 0

# Interpolate velocity data using cubic spatial interpolation
Interpolant = interpolant_unsteady(X, Y, U, V, method = "cubic")

Interpolant_u = Interpolant[0]
Interpolant_v = Interpolant[1]

# Compute gradient of the flow map $ \nabla F_{t_0}^t(\mathbf{x}_0) $

In [5]:
# Import function to compute gradient of flow map
from ipynb.fs.defs.gradient_flowmap import gradient_flowmap

# import package used to suppress output of integration of flow map. 
from IPython.utils import io

# Initial time (in days)
t0 = 0

# Final time (in days)
tN = 25

# Time step-size (in days)
dt = .1

# NOTE: For computing the backward trajectories set: tN < t0 and dt < 0.

time = np.arange(t0, tN+dt, dt) # len(time) = N

# auxiliary grid
aux_grid = [0.01, 0.01]

# initial x/y-coordinate
x0 = 3
y0 = -30

#Initial conditions
X0 = np.array([x0, y0])

# Compute gradient of flow map
with io.capture_output() as captured:
    gradFmap = gradient_flowmap(time, X0, X, Y, Interpolant_u, Interpolant_v, periodic, defined_domain, bool_unsteady, time_data, aux_grid)

# gradFmap stores the gradient of the flow map for all t in [t0,tN] 
# for the trajectory starting at the initial condition defined above.
# We extract gradient of flow map over the time-interval [t0,tN]
gradFmap_t0_tN = gradFmap[-1,:,:,0]

# Polar Decomposition of $ \nabla F_{t_0}^t(\mathbf{x}_0) $

In [6]:
# import function to compute PD
from ipynb.fs.defs.PD import PD

# import package to print latex statement
from IPython.display import display, Markdown

R, U, V = PD(gradFmap_t0_tN)

display(Markdown(
   rf"""
   
   $\nabla \mathbf{{F}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {gradFmap_t0_tN[0,0]:5.7} & {gradFmap_t0_tN[0,1]:5.7} \\ {gradFmap_t0_tN[1,0]:5.7} & {gradFmap_t0_tN[1,1]:5.7} \end{{pmatrix}} $
   
   $\mathbf{{R}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {R[0,0]:5.7} & {R[0,1]:5.7} \\ {R[1,0]:5.7} & {R[1,1]:5.7} \end{{pmatrix}} $
   
   $\mathbf{{U}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {U[0,0]:5.7} & {U[0,1]:5.7} \\ {U[1,0]:5.7} & {U[1,1]:5.7} \end{{pmatrix}} $
   
   $\mathbf{{V}}_{{t_0}}^{{t_N}}(\mathbf{{x}}_0)$: $\begin{{pmatrix}} {V[0,0]:5.7} & {V[0,1]:5.7} \\ {V[1,0]:5.7} & {V[1,1]:5.7} \end{{pmatrix}} $
   
   """))


   
   $\nabla \mathbf{F}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} -0.1763499 & 0.5995241 \\ -2.433198 & 2.421512 \end{pmatrix} $
   
   $\mathbf{R}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 0.5950052 & 0.8037218 \\ -0.8037218 & 0.5950052 \end{pmatrix} $
   
   $\mathbf{U}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 1.850685 & -1.589502 \\ -1.589502 & 1.922663 \end{pmatrix} $
   
   $\mathbf{V}_{t_0}^{t_N}(\mathbf{x}_0)$: $\begin{pmatrix} 0.3769215 & 0.4984563 \\ 0.4984563 & 3.396427 \end{pmatrix} $
   
   

# References

[1] Notebook 2.3.2. in "Transport Barriers and Coherent Structures in Flow Data" by Prof. George Haller.